In [26]:
from b2heavy.TwoPointFunctions.types2pts  import CorrelatorIO, Correlator
from b2heavy.TwoPointFunctions.fitter     import CorrFitter
from b2heavy.TwoPointFunctions.utils      import NplusN2ptModel, PeriodicExpDecay

In [27]:
import copy
import itertools
import jax
import autograd
import jax.numpy         as jnp
import autograd.numpy    as anp
import numpy             as np
import gvar              as gv

from scipy.linalg import issymmetric

In [28]:
ens = 'Coarse-1'
mes = 'Dst'
mom = '200'
binsize = 11

data_dir = '/Users/pietro/code/data_analysis/BtoD/Alex/'

io     = CorrelatorIO(ens,mes,mom,PathToDataDir=data_dir)
corr   = Correlator(io,jkBin=binsize)

In [29]:
NEXC = 3
TLIM = (10,17)
NT   = corr.Nt

SMR = ['1S-1S','d-d','d-1S']
POL = corr.data.polarization.values

In [30]:
Kdata = sorted(itertools.product(SMR,POL))
Kdata

[('1S-1S', 'Bot'),
 ('1S-1S', 'Par'),
 ('d-1S', 'Bot'),
 ('d-1S', 'Par'),
 ('d-d', 'Bot'),
 ('d-d', 'Par')]

# Custom fit

In [31]:
fitter = CorrFitter(corr,smearing=['d-d','1S-1S','d-1S'])

(X,meff,aeff), MEFF,AEFF, m_pr, apr = corr.EffectiveCoeff(trange=TLIM,covariance=True)

fitter.fit(
    Nstates           = NEXC,
    trange            = TLIM,
    verbose           = True,
    priors            = fitter.set_priors_phys(NEXC,Meff=MEFF,Aeff=AEFF),
    # scale_covariance  = True,
    # shrink_covariance = True,
    covariance        = False,
    override          = True
)

fit = fitter.fits[NEXC,TLIM]

/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in log
  return f_raw(*args, **kwargs)


1.1452(13)
---------- 3+3 fit in (10, 17) for mes: Dst of ens: Coarse-1 for mom: 200 --------------
 De-augmented chi2/ndof [ndof] = 1.4 [11] with p-value = 0.1842642123184468
Least Square Fit:
  chi2/dof [dof] = 0.4 [48]    Q = 1    logGBF = 864.28

Parameters:
            E 0   1.1448 (12)     [ 1.1452 (13) ]  
              1    -2.62 (40)     [  -1.81 (58) ]  *
              2    -0.99 (61)     [  -1.09 (80) ]  
              3    -3.5 (2.2)     [  -2.4 (2.5) ]  
              4    -2.1 (2.4)     [  -2.4 (2.5) ]  
              5    -1.3 (1.4)     [  -2.4 (2.5) ]  
     Z_1S_Bot 0    0.189 (11)     [   0.13 (56) ]  
              1    -0.21 (56)     [  -1.2 (1.2) ]  
              2     0.4 (1.3)     [   0.5 (1.5) ]  
              3     0.4 (1.1)     [   0.5 (1.5) ]  
              4     0.4 (2.9)     [   0.5 (3.0) ]  
              5     0.3 (2.9)     [   0.5 (3.0) ]  
     Z_1S_Par 0    0.268 (14)     [   0.26 (45) ]  
              1    -1.0 (1.1)     [  -1.2 (1.2) ]  
        

# Gradient with `jax`

In [ ]:
from b2heavy.TwoPointFunctions.fitter_alt import StagFitter

In [75]:
def ExpDecay(Nt, osc=False):
    return lambda t,E,Z : Z * (jnp.exp(-E*t) + jnp.exp(-E*(Nt-t))) * ((-1)**(t+1) if osc else 1.)

def StagDecay(Nt,Nexc):
    def model(t,E0,Z0a,Z0b, E1,Z1a,Z1b, *high):
        C_t  = ExpDecay(Nt)         ( t, E0, Z0a * Z0b )
        C_t += ExpDecay(Nt,osc=True)( t, E1, Z1a * Z1b )
        for n in range(2*Nexc-2):
            en, zn = high[2*n], high[2*n+1]**2
            C_t += ExpDecay(Nt,osc=(False if n%2==0 else True))( t, en, zn)
        return C_t
    return model

def Energies(Evec):
    '''
        `Energies(Evec)`

        Returns a vector `erg` of the same size of `Evec` with:
        - `erg[0] = Evec[0]`
        - `erg[1] = jnp.exp(Evec[1]) + erg[0]` 
        - `erg[2] = jnp.exp(Evec[2]) + erg[0]` 
        - `erg[3] = jnp.exp(Evec[3]) + erg[1]` 
        - `erg[4] = jnp.exp(Evec[3]) + erg[0]` 
        - `erg[5] = jnp.exp(Evec[3]) + erg[1]` 
    '''
    erg = [Evec[0]]
    for n in range(1,len(Evec)):
        erg.append(jnp.exp(Evec[n]) + erg[n-(1 if n==1 else 2)])
    return erg




def PeriodicExpDecay(Nt):
    return lambda t,E,Z: Z * ( jnp.exp(-E*t) + jnp.exp(-E*(Nt-t)) ) 

def NplusN2ptModel(Nstates,Nt,sm,pol):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def aux(t,p):
        E0, E1 = p['E'][0], p['E'][0] + jnp.exp(p['E'][1])
        Z0 = jnp.exp(p[f'Z_{sm1}_{pol}'][0]) * jnp.exp(p[f'Z_{sm2}_{pol}'][0])
        Z1 = jnp.exp(p[f'Z_{sm1}_{pol}'][1]) * jnp.exp(p[f'Z_{sm2}_{pol}'][1])
        ans = PeriodicExpDecay(Nt)(t,E0,Z0) + PeriodicExpDecay(Nt)(t,E1,Z1) * (-1)**(t+1)

        Es = [E0,E1]
        for i in range(2,2*Nstates):
            Ei = Es[i-2] + jnp.exp(p['E'][i])
            Z = p[f'Z_{sm if mix else sm1}_{pol}'][i-2 if mix else i]**2
            ans += PeriodicExpDecay(Nt)(t,Ei,Z) * (-1)**(i*(t+1))

            Es.append(Ei)
        return ans

    return aux



class StagFitter(Correlator):
    def __init__(self, io:CorrelatorIO, smearing=None, polarization=None, **kwargs):
        bsize = kwargs.get('jkBin')
        super().__init__(io,jkBin=bsize)

        self.smr = smearing     if not smearing==None     else list(self.data.smearing.values)
        self.pol = polarization if not polarization==None else list(self.data.polarization.values)
        self.keys = sorted(itertools.product(self.smr,self.pol))

    def Kpars(self,Nexc):
        '''
            Builds all the keys for dictionary of parameters and returns a dictionary:
            - keys are `(k,n)` where `k` is the name of the coefficient and the `n` is the index in the corresponding vector
            - values are the index number of the corresponding element in a flattened array of parameters.

            E.g.
            #          k name   idx       flattened_idx
                k = (    E     , 0)  ---> 0 
                k = ( Z_1S_Bot , 0)  ---> 1 
                k = ( Z_1S_Par , 0)  ---> 2 
                k = ( Z_d_Bot  , 0)  ---> 3 
                k = ( Z_d_Par  , 0)  ---> 4 
                k = (    E     , 1)  ---> 5 
                k = ( Z_1S_Bot , 1)  ---> 6 
                k = ( Z_1S_Par , 1)  ---> 7 
                k = ( Z_d_Bot  , 1)  ---> 8 
                k = ( Z_d_Par  , 1)  ---> 9 
                k = (    E     , 2)  ---> 10
                k = ( Z_1S_Bot , 2)  ---> 11
                k = ( Z_1S_Par , 2)  ---> 12
                k = (Z_d-1S_Bot, 0)  ---> 13
                k = (Z_d-1S_Par, 0)  ---> 14
                k = ( Z_d_Bot  , 2)  ---> 15
                k = ( Z_d_Par  , 2)  ---> 16
                k = (    E     , 3)  ---> 17
                k = ( Z_1S_Bot , 3)  ---> 18
                k = ( Z_1S_Par , 3)  ---> 19
                k = (Z_d-1S_Bot, 1)  ---> 20
                k = (Z_d-1S_Par, 1)  ---> 21
                k = ( Z_d_Bot  , 3)  ---> 22
                k = ( Z_d_Par  , 3)  ---> 23
        '''
        keys = []
        for n in range(2*Nexc):
            keys.append(('E',n))
            for (sm,pol) in self.keys:
                sm1,sm2 = sm.split('-')
                mix = sm1!=sm2

                k = f'Z_{sm if mix else sm1}_{pol}'
                if not (mix and n<2):
                    keys.append((k,n-2 if mix else n))


        return {k: i for i,k in enumerate(keys)}

    def pars_from_dict(self, pdict):
        '''
            Given a dictionary of parameters `pdict` e.g.
                `pdict['E']          = [ 1.14478779, -2.61763813, -0.99322927, -3.50597109, -2.09325374, -1.25464778]`
                `pdict['Z_1S_Unpol'] = [ 0.18893983, -0.20667998,  0.4060087 ,  0.37910282,  0.40034158,  0.27345506]  `
                ...
            returns an array of all elements in pdict, flattened according to the mapping of `self.Kpars`
        '''
        erg = Energies(pdict['E'])
        Nexc = len(erg)//2

        kidx = self.Kpars(Nexc)

        flatpar = [None]*len(np.concatenate(pdict.values()))
        for (k,n),i in kidx.items():
            if k=='E':
                flatpar[i] = erg[n]
            elif k.startswith('Z') and n<2 and '-' not in k:
                flatpar[i] = jnp.exp(pdict[k][n])
            else:
                flatpar[i] = pdict[k][n]

        return jnp.array(flatpar)

    def scalar_model(self, Nexc):
        '''
            It returns a differentiable version of the correlator function `model(tau,*params)` where
            - `tau` is the timeslice multiplied by 10**i, i being the position index of the key (`smr`,`pol`) in `self.keys`. E.g.
                ('1S-1S','Unpol') ---> tau = [11  , 12  , 13  , 14  , 15  , 16  , 17  ]
                ('d-1S' ,'Unpol') ---> tau = [110 , 120 , 130 , 140 , 150 , 160 , 170 ]
                ('d-d'  ,'Unpol') ---> tau = [1100, 1200, 1300, 1400, 1500, 1600, 1700]
            - `params` is the flattened vector of parameters. See documentation of `Kpars` and `pars_from_dict`
        '''

        kidx = self.Kpars(Nexc)

        def _model(tau, *params):
            i_sp   = int(jnp.log10(tau))-1
            t = tau/10**i_sp

            sm,pol = self.keys[i_sp] 
            sm1,sm2 = sm.split('-')
            mix = sm1!=sm2

            # Fundamental physical state
            e0  = params[kidx['E',0]]
            z0a = params[kidx[f'Z_{sm1}_{pol}',0]]
            z0b = params[kidx[f'Z_{sm2}_{pol}',0]]

            # Oscillating physical state
            e1  = params[kidx['E',1]]
            z1a = params[kidx[f'Z_{sm1}_{pol}',1]]
            z1b = params[kidx[f'Z_{sm2}_{pol}',1]]

            # Excited stated
            high = []
            for n in range(2,2*Nexc):
                en = params[kidx['E',n]]
                Zn = params[kidx[f'Z_{sm if mix else sm1}_{pol}',n-2 if mix else n]]
                high.extend([en,Zn])

            return StagDecay(self.Nt, Nexc)(t,e0,z0a,z0b,e1,z1a,z1b,*high)

        return _model

    def diff_model(self, Nexc, trange, wmat=None, **kwargs):
        (xdata,ydata) = self.format(
            trange       = trange,
            smearing     = self.smr,
            polarization = self.pol,
            flatten      = False,
            **kwargs
        )

        # xfit   = np.concatenate([xdata[k]*10**i for i,k in enumerate(self.keys)])
        yfit   = np.concatenate([ydata[k]       for   k in self.keys])

        w      = wmat if wmat is not None else jnp.linalg.inv(gv.evalcov(yfit))
        U,S,Vh = np.linalg.svd(w,hermitian=True)
        W      = U @ np.diag(np.sqrt(S)) @ Vh

        def _model(pdict):  
            return jnp.concatenate(
                [NplusN2ptModel(Nexc,self.Nt,smr,pol)(xdata[smr,pol],pdict) for smr,pol in self.keys]
            )

        def _jac(pdict):
            return jax.jacfwd(_model)(pdict)

        def _hes(pdict):
            return jax.jacfwd(jax.jacrev(_model))(pdict)


        return _model, _jac, _hes


In [85]:
self = StagFitter(
    io       = io,
    jkBin    = binsize,
    smearing = ['1S-1S','d-d','d-1S']
)

# model, residual = self.diff_model(NEXC,TLIM)
# p = self.pars_from_dict(gv.mean(fit.p))


f = self.diff_model(NEXC,TLIM)

# print(f(fit.pmean))


jac = jax.jacfwd(f)(dict(fit.pmean))

In [91]:
jac['Z_1S_Par']

Array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.64902444e-05, -1.47197284e-06,  4.317

# Gradient computation

In [17]:
def Energies(Evec):
    erg = [Evec[0]]
    for n in range(1,len(Evec)):
        erg.append(anp.exp(Evec[n]) + erg[n-(1 if n==1 else 2)])
    return erg

In [18]:
Kpars = []
for n in range(2*NEXC):
    Kpars.append(('E',n))

    for (sm,pol) in sorted(itertools.product(SMR,POL)):
        sm1,sm2 = sm.split('-')
        mix = sm1!=sm2
        k = f'Z_{sm if mix else sm1}_{pol}'

        if not (mix and n<2):
            Kpars.append((k,n-2 if mix else n))

kid   = {k:i for i,k in enumerate(Kpars)}


In [19]:
len(np.concatenate(fit.pmean.values()))

38

In [9]:
erg = Energies(fit.pmean['E'])

pVector = [None]*len(Kpars)
diffpar = [None]*len(Kpars)
for (k,n),i in kid.items():
    pVector[i] = fit.p[k][n]
    if k=='E':
        diffpar[i] = erg[n]
    elif k.startswith('Z') and n<2 and '-' not in k:
        diffpar[i] = np.exp(fit.pmean[k][n])
    else:
        diffpar[i] = fit.pmean[k][n]

    print(f'{i:<2}     fit.p[{k:^12}][{n}]     {fit.p[k][n]:<10}  {pVector[i]:>10}  {round(diffpar[i],8):>10}')
    


NameError: name 'fit' is not defined

In [10]:
?autograd.jacobian

Signature: autograd.jacobian(fun, argnum=0, *nary_op_args, **nary_op_kwargs)
Docstring:
Returns a function which computes the Jacobian of `fun` with respect to
positional argument number `argnum`, which must be a scalar or array. Unlike
`grad` it is not restricted to scalar-output functions, but also it cannot
take derivatives with respect to some argument types (like lists or dicts).
If the input to `fun` has shape (in1, in2, ...) and the output has shape
(out1, out2, ...) then the Jacobian has shape (out1, out2, ..., in1, in2, ...).
File:      ~/opt/anaconda3/envs/stat/lib/python3.11/site-packages/autograd/wrap_util.py
Type:      function

In [25]:
def ExpDecay(Nt, osc=False):
    return lambda t,E,Z : Z * (anp.exp(-E*t) + anp.exp(-E*(Nt-t))) * ((-1)**(t+1) if osc else 1.)

def StaggDecay2(Nt,Nexc):

    def model(t,E0,Z0a,Z0b, E1,Z1a,Z1b, *high):
        C_t  = ExpDecay(Nt)         ( t, E0, anp.exp(Z0a)*anp.exp(Z0b) )
        C_t += ExpDecay(Nt,osc=True)( t, E1, anp.exp(Z1a)*anp.exp(Z1b) )
        for n in range(2*Nexc-2):
            en, zn = high[2*n], high[2*n+1]**2
            C_t += ExpDecay(Nt,osc=(False if n%2==0 else True))( t, en, zn)
        return C_t

    return model

def StaggDecay3(Nt,Nexc):

    def model(t,E0,Z0a,Z0b, E1,Z1a,Z1b, *high):
        C_t  = ExpDecay(Nt)         ( t, E0, Z0a * Z0b )
        C_t += ExpDecay(Nt,osc=True)( t, E1, Z1a * Z1b )
        for n in range(2*Nexc-2):
            en, zn = high[2*n], high[2*n+1]**2
            C_t += ExpDecay(Nt,osc=(False if n%2==0 else True))( t, en, zn)
        return C_t

    return model

In [98]:
Kdataa

array([['1S-1S', 'Bot'],
       ['1S-1S', 'Par'],
       ['d-1S', 'Bot'],
       ['d-1S', 'Par'],
       ['d-d', 'Bot'],
       ['d-d', 'Par']], dtype='<U5')

In [26]:
Kdataa = anp.array(sorted(itertools.product(SMR,POL)))

def flat_pars_model_scalar2(tau,*params):  
    i_sp   = int(anp.log10(tau))-1
    t = tau/10**i_sp

    sm,pol = Kdataa[i_sp] 
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    # Fundamental physical state
    e0  = params[kid['E',0]]
    z0a = params[kid[f'Z_{sm1}_{pol}',0]]
    z0b = params[kid[f'Z_{sm2}_{pol}',0]]

    # Oscillating physical state
    e1  = anp.exp(params[kid['E',1]]) + e0
    z1a = params[kid[f'Z_{sm1}_{pol}',1]]
    z1b = params[kid[f'Z_{sm2}_{pol}',1]]

    erg = [e0,e1]

    # Excited stated
    high = []
    for n in range(2,2*NEXC):
        en = anp.exp(params[kid['E',n]]) + erg[n-2]
        Zn = params[kid[f'Z_{sm if mix else sm1}_{pol}',n-2 if mix else n]]
        high.extend([en,Zn])

        erg.append(en)
    
    return StaggDecay2(NT,NEXC)(t,e0,z0a,z0b,e1,z1a,z1b,*high)

def flat_pars_model_scalar3(tau,*params):  
    i_sp   = int(anp.log10(tau))-1
    t = tau/10**i_sp

    sm,pol = Kdataa[i_sp] 
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    # Fundamental physical state
    e0  = params[kid['E',0]]
    z0a = params[kid[f'Z_{sm1}_{pol}',0]]
    z0b = params[kid[f'Z_{sm2}_{pol}',0]]

    # Oscillating physical state
    e1  = params[kid['E',1]]
    z1a = params[kid[f'Z_{sm1}_{pol}',1]]
    z1b = params[kid[f'Z_{sm2}_{pol}',1]]



    # Excited stated
    high = []
    for n in range(2,2*NEXC):
        en = params[kid['E',n]]
        Zn = params[kid[f'Z_{sm if mix else sm1}_{pol}',n-2 if mix else n]]
        high.extend([en,Zn])
    


    return StaggDecay3(NT,NEXC)(t,e0,z0a,z0b,e1,z1a,z1b,*high)

In [65]:
xdata = [x*10**i for i,x in enumerate(fit.x)]
for xr in xdata:
    for x in xr:
        flat_pars_model_scalar3(x,*pVector)
xdatad = {tuple(k):xdata[i] for i,k in enumerate(Kdataa)}

In [36]:
phi2 = np.array([flat_pars_model_scalar2(t,*pVector) for t in np.concatenate(xdata)])
phi = np.array([flat_pars_model_scalar3(t,*diffpar) for t in np.concatenate(xdata)])

assert np.allclose(gv.mean(phi2),phi,rtol=1e-15)

Calculate the gradient matrix
$$
    \Delta_{i \alpha} = \frac{\partial}{\partial a^\alpha} \phi(t_i;\bar a) =: \phi_\alpha(t_i) \qquad \in (N_t \times N_\text{pars})
$$

In [37]:
# grads = {(k,n): autograd.grad(flat_pars_model_scalar2,argnum=(kid[k,n]+1)) for (k,n),i in kid.items()}
grads = {(k,n): autograd.grad(flat_pars_model_scalar3,argnum=(kid[k,n]+1)) for (k,n),i in kid.items()}

Let's make a check. The derivative 
$$
\frac{\partial \phi(t)}{\partial Z_0^{(1S,Bot)}} = \begin{cases} 2Z_0^{(1S,\text{Bot})}(e^{-E_0t}+e^{-E_0(N_t-t)}) \text{ for } t \in t_{(1S-1S,\text{Bot})} \\ 2Z_0^{(d,\text{Bot})}(e^{-E_0t}+e^{-E_0(N_t-t)})\text{ for } t \in t_{(d-1S,\text{Bot})} \\ 0 \text{ elsewhere }  \end{cases}
$$

In [72]:
t = xdatad['1S-1S','Bot']

dcdz = 2*np.exp(fit.pmean['Z_1S_Bot'][0]) * (np.exp(- fit.pmean['E'][0] * t ) + np.exp(- fit.pmean['E'][0] * (NT-t) ))
autod = np.array([grads[('Z_1S_Bot',0)](x,*diffpar) for x in t])
assert np.allclose(dcdz,autod,atol=0.,rtol=1e-12)
autod - dcdz

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [73]:
g = []
for (k,n),i in kid.items():
    g.append(np.array([grads[k,n](t,*diffpar) for t in np.concatenate(xdata)]))
g = np.array(g).T

/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


# Checks

In [74]:
def issymm(M,rtol=1e-12,atol=0.):
    return np.allclose(M,M.T,atol=atol,rtol=rtol)

def rdist(M,N):
    tol = 1e-30
    while True:
        if np.allclose(M,N,atol=0.,rtol=tol):
            break
        else:
            tol *= 10
    return 10**np.log10(tol)

We have to use now the matrix of weights used to perform the fit, i.e. minimize
$$
    \chi^2 = || W \cdot (\vec y - \vec \phi)||^2 = {}^t (\vec y - \vec \phi)\cdot w \cdot (\vec y - \vec \phi) 
$$
where $w = {}^tW W = W^2$

In [75]:
w = np.linalg.inv(gv.evalcov(fit.y))
W = np.linalg.cholesky(w)

In [76]:
res = gv.mean(phi) - gv.mean(fit.y)
assert fit.chi2red == res.T @ w @ res

In [77]:
fit.chi2red

15.17348543665168

$$
    H_{\alpha\beta} = (W\cdot\phi^\alpha,W\cdot\phi^\beta) = \phi^\alpha\cdot w\cdot \phi^\beta
$$

In [78]:
Hmat = g.T @ w @ g
Hinv = np.linalg.inv(Hmat)

$$
\mathcal P = W \phi^\alpha \,(H^{-1})^{\alpha\beta}\, {}^t \phi^\beta W
$$

In [79]:
Wg = W @ g
Proj = Wg @ ((Hinv+Hinv.T)/2.) @ Wg.T

In [84]:
np.trace(Proj)

37.99999990301794

In [83]:
rdist(Proj@Proj,Proj)

0.10000000000000005

In [81]:
xx = (np.eye(W.shape[0])-Proj) @ W @ res
(xx.T @ xx)

2.2468850196018364

# Junks

In [14]:
w = np.linalg.inv(gv.evalcov(fit.y))


$$
\mathcal C_{\alpha\beta} = (H^{-1})^{\alpha\alpha'} \phi^{\alpha'} w \mathcal C w \phi^\beta (H^{-1})^{\beta'\beta}
$$

In [85]:
Hmat = g.T @ w @ g
Hinv = np.linalg.inv(Hmat)

In [86]:
wg = w @ g
proj = w - wg @ Hinv @ wg.T

In [87]:
xdata,ydata = corr.format(trange=TLIM,flatten=True,covariance=True,smearing=SMR)
cov = gv.evalcov(ydata)

In [88]:
cov_pars = Hinv @ (wg.T @ cov @ wg) @ Hinv
gv.gvar(gv.mean(pVector),cov_pars)

ValueError: non-symmetric covariance matrix:
[[2.51916892e-03 2.80756683e-02 3.19151990e-02 ... 1.13147446e-01
  4.29933861e-02 4.36840972e-02]
 [2.80756682e-02 3.17436761e-01 3.59237178e-01 ... 1.23440762e+00
  4.98205364e-01 4.77801440e-01]
 [3.19151987e-02 3.59237177e-01 4.13439585e-01 ... 1.39760574e+00
  4.94733993e-01 5.39703688e-01]
 ...
 [1.13147446e-01 1.23440760e+00 1.39760572e+00 ... 3.45997801e+01
  9.04728683e+00 1.06022209e+01]
 [4.29933850e-02 4.98205355e-01 4.94733954e-01 ... 9.04728684e+00
  4.28920714e+00 2.88089478e+00]
 [4.36840985e-02 4.77801449e-01 5.39703714e-01 ... 1.06022212e+01
  2.88089446e+00 3.50084234e+00]]

In [89]:
chi_exp = np.trace(proj @ cov)
chi_exp

2.793127312228611

# Alternative implementation

In [107]:
def Energies(Evec):
    erg = [Evec[0]]
    for n in range(1,len(Evec)):
        erg.append(anp.exp(Evec[n]) + erg[n-(1 if n==1 else 2)])
    return erg

In [109]:
Kdataa = np.array(sorted(itertools.product(SMR,POL)))

def flat_pars_model_scalar(tau,*params):  
    i_sp   = int(anp.log10(tau))-1
    t = tau/10**i_sp

    sm,pol = Kdataa[i_sp] 
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    erg = Energies([params[kid['E',n]] for n in range(2*NEXC)])
    Z0 = anp.exp(params[kid[f'Z_{sm1}_{pol}',0]]) * anp.exp(params[kid[f'Z_{sm2}_{pol}',0]])
    Z1 = anp.exp(params[kid[f'Z_{sm1}_{pol}',1]]) * anp.exp(params[kid[f'Z_{sm2}_{pol}',1]])
    
    C_t = ExpDecay(NT)(t,erg[0],Z0) + ExpDecay(NT,osc=True)(t,erg[1],Z1)

    for n in range(2,2*NEXC):
        Zn = params[kid[f'Z_{sm if mix else sm1}_{pol}',n-2 if mix else n]] ** 2
        C_t += ExpDecay(NT,osc=(False if n%2==0 else True))(t,erg[n],Zn)

    return C_t

for xr in xdata:
    for x in xr:
        print(flat_pars_model_scalar(x,*pVector)-flat_pars_model_scalar2(x,*pVector))

0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
0 ± 0
